In [ ]:
!pip install numpy
!pip install msgpack
!pip install pandas

***Вариант №36***

Задание №1

In [1]:
import numpy as np

def process_npy(file_path, output_json_file, normalized_file):
    matrix = np.load(file_path)
    total_sum = np.sum(matrix)
    total_avg = np.mean(matrix)
    main_diagonal = np.diagonal(matrix)
    sum_main_diag = np.sum(main_diagonal)
    avg_main_diag = np.mean(main_diagonal)
    secondary_diagonal = np.fliplr(matrix).diagonal()
    sum_secondary_diag = np.sum(secondary_diagonal)
    avg_secondary_diag = np.mean(secondary_diagonal)
    max_val = np.max(matrix)
    min_val = np.min(matrix)

    open(output_json_file, "w").write(
        f"""
    {{
        "sum": {total_sum},
        "avr": {total_avg},
        "sumMD": {sum_main_diag},
        "avrMD": {avg_main_diag},
        "sumSD": {sum_secondary_diag},
        "avrSD": {avg_secondary_diag},
        "max": {max_val},
        "min": {min_val}
    }}
    """
    )

    min_matrix = matrix.min()
    max_matrix = matrix.max()
    normalized_matrix = (matrix - min_matrix) / (max_matrix - min_matrix)

    np.save(normalized_file, normalized_matrix)
    print(f"Нормализованная матрица сохранена в {normalized_file}")


normalized_file = "first_task_normalized.npy"
file_path = "first_task.npy"
update_json_file = "first_task_results.json"
folder = "./data/"
process_npy(folder + file_path, folder + update_json_file, folder + normalized_file)

Нормализованная матрица сохранена в ./data/first_task_normalized.npy


Задание №2

In [43]:
import numpy as np
import os

folder = "./data/"

matrix = np.load(folder + file_path)
indices_i = []
indices_j = []
values = []

for i in range(matrix.shape[0]):
    for j in range(matrix.shape[1]):
        if matrix[i, j] > matrix[i, (j + 1) % matrix.shape[1]]:
            indices_i.append(i)
            indices_j.append(j)
            values.append(matrix[i, j])

indices_i = np.array(indices_i)
indices_j = np.array(indices_j)
values = np.array(values)

np.savez(folder + 'second_arrays.npz', indices_i=indices_i, indices_j=indices_j, values=values)
np.savez_compressed(folder + 'second_arrays_compressed.npz', indices_i=indices_i, indices_j=indices_j, values=values)
size_uncompressed = os.path.getsize(folder + "second_arrays.npz")
size_compressed = os.path.getsize(folder + 'second_arrays_compressed.npz')

print(f"Size of uncompressed npz file: {size_uncompressed} bytes")
print(f"Size of compressed npz file: {size_compressed} bytes")

Size of uncompressed npz file: 306226 bytes
Size of compressed npz file: 37760 bytes


Задание №3

In [ ]:
import msgpack
import pandas as pd
import os


def process_npy(folder, file_json_path, output_json_file, output_msgpack_file):
    data = pd.read_json(folder + file_json_path)
    aggregated_data = (
        data.groupby("name")
        .agg(
            avg_price=("price", "mean"),
            max_price=("price", "max"),
            min_price=("price", "min"),
        )
        .reset_index()
    )

    aggregated_data.to_json(folder + output_json_file, force_ascii=False, orient='records')

    with open(folder + output_msgpack_file, 'wb') as f:
        packed = msgpack.packb(
            aggregated_data.to_dict(orient="records"), use_bin_type=True
        )
        f.write(packed)

    size_json = os.path.getsize(folder + output_json_file)
    size_msgpack = os.path.getsize(folder + output_msgpack_file)

    print(f"Size of JSON file: {size_json} bytes")
    print(f"Size of Msgpack file: {size_msgpack} bytes")

folder = "./data/"
file_json_path = "third_task.json"
update_json_file = "third_task_results.json"
output_msgpack_file = "third_task_results.msgpack"
process_npy(folder, file_json_path, update_json_file, output_msgpack_file)

Size of JSON file: 7379 bytes
Size of Msgpack file: 5891 bytes


Задание №4

In [ ]:
import pickle
import pandas as pd

method = {
        "add": lambda x, y: x + y,
        "sub": lambda x, y: x - y,
        "percent+": lambda x, y: x * (1 + (float(y) / 100)),
        "percent-": lambda x, y: x * (1 - (float(y) / 100)),
}


def process_npy(folder, file_pkl_path, update_json_file, output_pkl_file):
    products = pickle.load(open(folder + file_pkl_path, "rb"))
    updates = pd.read_json(folder + update_json_file)
    for _, row in updates.iterrows():
        product_name = row['name']
        if product_name in products:
            products[product_name]['price'] = method[row['method']](products[product_name]['price'], row['param'])
    with open(folder + output_pkl_file, 'wb') as f:
        pickle.dump(products, f)

folder = "./data/"
file_pkl_path = "fourth_task_products.pkl"
update_json_file = "fourth_task_updates.json"
output_pkl_file = "fourth_task_products_results.pkl"
process_npy(folder, file_pkl_path, update_json_file, output_pkl_file)

Задание №5

In [ ]:
import json
import pickle
import msgpack
import pandas as pd
from numpy import int64, float64

def process_npy(folder, file_csv_path, output_csv_file, output_json_file, output_msgpack_file, output_pkl_file):
    df = pd.read_csv(folder + file_csv_path)
    selected_fields = ['hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children']
    df_selected = df[selected_fields]
    numerical_fields = df_selected.select_dtypes(include=[int64, float64]).columns
    stats = {}
    for field in numerical_fields:
        stats[field] = {
            'max': df_selected[field].max(),
            'min': df_selected[field].min(),
            'mean': df_selected[field].mean(),
            'sum': df_selected[field].sum(),
            'std': df_selected[field].std()
        }
    categorical_fields = df_selected.select_dtypes(include=[object]).columns
    for field in categorical_fields:
        stats[field] = df_selected[field].value_counts().to_dict()

    def convert_to_serializable(obj):
        if isinstance(obj, int64):
            return int(obj)
        if isinstance(obj, float64):
            return float(obj)
        return obj

    with open(folder + output_json_file, 'w') as f:
        json.dump(stats, f, ensure_ascii=False, indent=4, default=convert_to_serializable)

    # Save the selected DataFrame to different formats
    df_selected.to_csv(folder + output_csv_file, index=False)
    df_selected.to_json(folder + output_json_file, orient='records', force_ascii=False)
    with open(folder + output_msgpack_file, 'wb') as f:
        packed = msgpack.packb(df_selected.to_dict(orient='records'), use_bin_type=True)
        f.write(packed)
    with open(folder + output_pkl_file, 'wb') as f:
        pickle.dump(df_selected, f)

    size_csv = os.path.getsize(folder + output_csv_file)
    size_json = os.path.getsize(folder + output_json_file)
    size_msgpack = os.path.getsize(folder + output_msgpack_file)
    size_pkl = os.path.getsize(folder + output_pkl_file)

    print(f"Size of CSV file: {size_csv} bytes")
    print(f"Size of JSON file: {size_json} bytes")
    print(f"Size of Msgpack file: {size_msgpack} bytes")
    print(f"Size of PKL file: {size_pkl} bytes")



folder = "./data/"
file_csv_path = "hotel_booking.csv"
output_csv_file = "five_results.csv"
output_json_file = "five_results.json"
output_msgpack_file = "five_results.msgpack"
output_pkl_file = "five_results.pkl"
process_npy(folder, file_csv_path, output_csv_file, output_json_file, output_msgpack_file, output_pkl_file)

Size of CSV file: 4594853 bytes
Size of JSON file: 26204301 bytes
Size of Msgpack file: 22696794 bytes
Size of PKL file: 8836234 bytes
